<a href="https://colab.research.google.com/github/pgurazada/explore-dinov2/blob/main/emotion_classification_dinov2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this exercise, we train a classification model that uses the feature extraction capabilities of [Dino V2](https://arxiv.org/pdf/2304.07193.pdf).

# Imports

In [1]:
!pip install -q datasets flaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [2]:
import os
import torch
import io

import numpy as np
import torchvision.transforms as T

from PIL import Image
from tqdm import tqdm
from datasets import load_dataset
from flaml import AutoML

In [3]:
!pip show torch torchvision

Name: torch
Version: 2.1.0+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision
---
Name: torchvision
Version: 0.16.0+cu118
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, requests, torch
Required-by: fastai


# Data

In [4]:
emotion_ds = load_dataset("sxdave/emotion_detection")

Resolving data files:   0%|          | 0/157 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/45 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
train_data = list(emotion_ds['train'])

In [6]:
test_data = list(emotion_ds['test'])

# Model for Embeddings

In [7]:
dinov2_vit14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:01<00:00, 248MB/s]


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
dinov2_vit14.to(device)

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [9]:
transform_image = T.Compose(
    [
        T.ToTensor(),
        T.Resize(244, antialias=True),
        T.CenterCrop(224),
        T.Normalize([0.5], [0.5])
    ]
)

In [10]:
def load_image(img: str) -> torch.Tensor:
    """
    Load an image and return a tensor that can be used as an input to DINOv2.
    """

    transformed_img = transform_image(img)[:3].unsqueeze(0)

    return transformed_img

In [11]:
def compute_embeddings(files: list) -> dict:
    """
    Create an index that contains all of the images in the specified list of files.
    """
    all_embeddings = {}

    with torch.no_grad():
      for i, file in enumerate(tqdm(files)):
        embeddings = dinov2_vit14(load_image(file['image']).to(device))

        all_embeddings[i] = np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()

    return all_embeddings

In [12]:
train_embeddings = compute_embeddings(train_data)

100%|██████████| 157/157 [00:12<00:00, 12.55it/s]


In [13]:
test_embeddings = compute_embeddings(test_data)

100%|██████████| 23/23 [00:00<00:00, 51.04it/s]


In [14]:
ytrain = np.array([file['label'] for file in train_data])
train_embedding_list = list(train_embeddings.values())
Xtrain = np.array(train_embedding_list).reshape(-1, dinov2_vit14.embed_dim)

ytest = np.array([file['label'] for file in test_data])
test_embedding_list = list(test_embeddings.values())
Xtest = np.array(test_embedding_list).reshape(-1, dinov2_vit14.embed_dim)

# Model for Classification

In [15]:
automl = AutoML()

In [16]:
automl.fit(
    X_train=Xtrain, y_train=ytrain,
    time_budget=240,
    log_file_name='emotions.log',
    task='classification',
    metric='accuracy',
    split_ratio=.3
)

[flaml.automl.logger: 12-13 14:03:18] {1679} INFO - task = classification
[flaml.automl.logger: 12-13 14:03:18] {1690} INFO - Evaluation method: cv
[flaml.automl.logger: 12-13 14:03:18] {1788} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 12-13 14:03:18] {1900} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 12-13 14:03:18] {2218} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 12-13 14:03:18] {2344} INFO - Estimated sufficient time budget=2820s. Estimated necessary time budget=65s.
[flaml.automl.logger: 12-13 14:03:18] {2391} INFO -  at 0.3s,	estimator lgbm's best error=0.4145,	best estimator lgbm's best error=0.4145
[flaml.automl.logger: 12-13 14:03:18] {2218} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 12-13 14:03:18] {2391} INFO -  at 0.5s,	estimator lgbm's best error=0.3831,	best estimator lgbm's best error=0.3831
[flaml.automl.logger: 12-13 14:03:

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 12-13 14:07:18] {2391} INFO -  at 240.0s,	estimator lrl1's best error=0.2188,	best estimator lrl1's best error=0.2188


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.automl.logger: 12-13 14:07:18] {2627} INFO - retrain lrl1 for 0.4s
[flaml.automl.logger: 12-13 14:07:18] {2630} INFO - retrained model: LogisticRegression(n_jobs=-1, penalty='l1', solver='saga')
[flaml.automl.logger: 12-13 14:07:18] {1930} INFO - fit succeeded
[flaml.automl.logger: 12-13 14:07:18] {1931} INFO - Time taken to find the best model: 240.04762935638428


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [17]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print(f'Best accuracy on validation data: {(1-automl.best_loss):.3f}')
print(f'Training duration of best run: {(automl.best_config_train_time):.4f} s')

Best ML leaner: lrl1
Best hyperparmeter config: {'C': 1.0}
Best accuracy on validation data: 0.781
Training duration of best run: 0.4214 s


In [18]:
automl.model.estimator

LogisticRegression(n_jobs=-1, penalty='l1', solver='saga')

In [19]:
ypred = automl.predict(Xtest)

In [20]:
(ypred == ytest).mean()

0.6521739130434783